In [2]:
import os, random, shutil, getpass, sys
import torch
import cv2
import matplotlib.pyplot as plt
import kagglehub


In [3]:
!nvidia-smi
print("torch cuda available:", torch.cuda.is_available())

Tue Dec 16 03:44:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
#run if local
# IN_COLAB = False

# import os, getpass

# required = ["README.md"]
# for f in required:
#     if not os.path.exists(f):
#         raise RuntimeError("Run this notebook from inside the repo folder (where README.md is).")

# #set local dataset folder
# DATA_ROOT = "/path/to/traffic_data"  # folder that contains train/val/test (or contains a subfolder that does)

# print("Local ready. CWD =", os.getcwd())
# print("DATA_ROOT =", DATA_ROOT)


In [5]:
#run if in colab

IN_COLAB = True

# GPU check
os.system("nvidia-smi")

#colab imports
from google.colab import drive, files

#mount Drive
drive.mount("/content/drive")

#upload kaggle.json
files.upload() #select kaggle.json

#install packages
os.system("pip -q install ultralytics opencv-python matplotlib kagglehub")

#put Kaggle creds where tools expect them
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.copy("kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 0o600)

#dataset in Drive
DATA_ROOT = "/content/drive/MyDrive/traffic_data"
os.makedirs(DATA_ROOT, exist_ok=True)

print("Colab mode ready. DATA_ROOT =", DATA_ROOT)


MessageError: Error: credential propagation was unsuccessful

In [ ]:
REPO_URL = "https://github.com/elizabethszent/Detecting-Road-Objects-for-Autonomous-Cars-in-Egypt.git"
REPO_DIR = "Detecting-Road-Objects-for-Autonomous-Cars-in-Egypt"

if not os.path.exists(REPO_DIR):
    os.system(f"git clone {REPO_URL}")

os.chdir(REPO_DIR)
os.system("ls")

In [ ]:
%cd /content/Detecting-Road-Objects-for-Autonomous-Cars-in-Egypt
!mkdir -p notebooks
!cp "/content/drive/MyDrive/Colab Notebooks/AutonomousCarsinEgypt.ipynb" "notebooks/AutonomousCarsinEgypt.ipynb"
!ls -lah notebooks

In [ ]:
os.system('git config user.name "elizabethszent"')
os.system('git config user.email "elizabethszentmiklossy@gmail.com"')

token = getpass.getpass("GitHub token (hidden): ")
os.system(f"git remote set-url origin https://{token}@github.com/elizabethszent/Detecting-Road-Objects-for-Autonomous-Cars-in-Egypt.git")

# write/update .gitignor
gitignore_text = """data/
runs/
kaggle.json
.ipynb_checkpoints/
__pycache__/
*.pyc
.env
.venv/
venv/
.DS_Store
"""
with open(".gitignore", "w") as f:
    f.write(gitignore_text)

# if kaggle.json was ever tracked, untrack it now
os.system("git rm --cached kaggle.json || true")

#add files
os.system("git add .")
os.system('git commit -m "update notebooks and config" || echo "Nothing to commit"')
os.system("git branch -M main")
os.system("git push -u origin main")


In [ ]:
#colab-only

IN_COLAB = "google.colab" in sys.modules
if not IN_COLAB:
    print("Not running in Colab — skipping.")
else:
    from google.colab import drive
    drive.mount("/content/drive")

    DRIVE_ROOT = "/content/drive/MyDrive/traffic_data"
    os.makedirs(DRIVE_ROOT, exist_ok=True)

    train_img_dir = os.path.join(DRIVE_ROOT, "train", "train", "images")
    train_lab_dir = os.path.join(DRIVE_ROOT, "train", "train", "labels")

    def nonempty_with_exts(p, exts):
        return os.path.isdir(p) and any(f.lower().endswith(exts) for f in os.listdir(p))

    has_train_imgs = nonempty_with_exts(train_img_dir, (".jpg", ".jpeg", ".png"))
    has_train_labs = nonempty_with_exts(train_lab_dir, (".txt",))

    if has_train_imgs and has_train_labs:
        print("Dataset already exists in Drive — skipping download+copy.")
    else:
        # install + import kagglehub in THIS cell (runtime resets wipe it)
        !pip -q install kagglehub
        import kagglehub

        cache_path = kagglehub.dataset_download("mohamedra9ab/object-detection-for-autonomous-cars-egypt")
        print("Downloaded to cache:", cache_path)

        # Copy contents into DRIVE_ROOT (no new dir)
        print("Copying dataset contents into:", DRIVE_ROOT)
        for item in os.listdir(cache_path):
            src = os.path.join(cache_path, item)
            dst = os.path.join(DRIVE_ROOT, item)

            if os.path.isdir(src):
                shutil.copytree(src, dst, dirs_exist_ok=True)
            else:
                os.makedirs(os.path.dirname(dst), exist_ok=True)
                shutil.copy2(src, dst)

        print("Copy complete.")

    # Debug printout (always)
    print("train/train/images exists:", os.path.isdir(train_img_dir), "count:",
          (len(os.listdir(train_img_dir)) if os.path.isdir(train_img_dir) else 0))
    print("train/train/labels exists:", os.path.isdir(train_lab_dir), "count:",
          (len(os.listdir(train_lab_dir)) if os.path.isdir(train_lab_dir) else 0))

    DATA_ROOT = DRIVE_ROOT
    print("DATA_ROOT set to:", DATA_ROOT)

In [ ]:
!find "/content/drive/MyDrive/traffic_data" -maxdepth 6 -type d -path "*/train/images" | head -n 20

In [ ]:
dataset_path = "/content/drive/MyDrive/traffic_data/object-detection-for-autonomous-cars-egypt"
print(os.path.isdir(os.path.join(dataset_path, "train", "train", "images")))
print(os.path.isdir(os.path.join(dataset_path, "train", "train", "labels")))
print(os.path.isdir(os.path.join(dataset_path, "test", "test", "images")))
print(os.path.isdir(os.path.join(dataset_path, "test", "test", "labels")))

In [ ]:
yaml_text = f"""path: {dataset_path}
train: train/train/images
val: train/train/images
test: test/test/images

names:
  0: car
  1: bus
  2: truck
  3: motor
  4: bike
  5: rider
  6: person
  7: tuktuk
  8: train
  9: traffic_sign
  10: traffic_light
  11: banners
"""
with open("egypt_traffic.yaml", "w") as f:
    f.write(yaml_text)

print(open("egypt_traffic.yaml").read())


In [ ]:
img_dir = os.path.join(dataset_path, "train/train/images")
lab_dir = os.path.join(dataset_path, "train/train/labels")

img_files = [f for f in os.listdir(img_dir) if f.lower().endswith((".jpg",".png",".jpeg"))]
img_name = random.choice(img_files)

img_path = os.path.join(img_dir, img_name)
lab_path = os.path.join(lab_dir, os.path.splitext(img_name)[0] + ".txt")

img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
h, w = img.shape[:2]

if os.path.exists(lab_path):
    with open(lab_path) as f:
        for line in f:
            c, xc, yc, bw, bh = map(float, line.split())
            x1 = int((xc - bw/2) * w); y1 = int((yc - bh/2) * h)
            x2 = int((xc + bw/2) * w); y2 = int((yc + bh/2) * h)
            cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,0), 2)

plt.figure(figsize=(10,6))
plt.imshow(img); plt.title(img_name); plt.axis("off")
plt.show()


In [ ]:
model = YOLO("yolov8s.pt")

project_dir = "/content/drive/MyDrive/yolo_runs" if IN_COLAB else "runs"
model.train(
    data="egypt_traffic.yaml",
    imgsz=640,
    epochs=50,
    batch=16,
    project=project_dir,
    name="yolov8s_baseline"
)